In [34]:

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv')

numrows = 25549 # can be as large as 25549

#create an empty dataframe to hold 100 values
heuristic_df = pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows,
                             'guess':[False]*numrows, #logical guess
                             'rain_tomorrow':[False]*numrows, #historical observation
                             'correct':[False]*numrows, #TRUE if your guess matches the historical observation
                             'true_positive':[False]*numrows, #TRUE If you said it would rain and it did
                             'false_positive':[False]*numrows,#TRUE If you sait id would rain and it didn't
                             'true_negative':[False]*numrows, #TRUE if you said it wouldn't rain and it didn't
                             'false_negative':[False]*numrows}) #TRUE if you said it wouldn't raing and it did

#sort columns for convience
seq = ['yesterday',
       'today',
       'tomorrow',
       'guess',
       'rain_tomorrow',
       'correct',
       'true_positive',
       'false_positive',
       'true_negative',
       'false_negative']
heuristic_df = heuristic_df.reindex(columns=seq)

In [35]:
df.head()

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


In [36]:
heuristic_df.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
0,0.0,0.0,0.0,False,False,False,False,False,False,False
1,0.0,0.0,0.0,False,False,False,False,False,False,False
2,0.0,0.0,0.0,False,False,False,False,False,False,False
3,0.0,0.0,0.0,False,False,False,False,False,False,False
4,0.0,0.0,0.0,False,False,False,False,False,False,False


In [37]:
# here is an example loop that populates the dataframe created earlier
# with the total percip from yesterday and today
# then the guess is set to true if rained both yesterday and today 
for z in range(numrows):
    #start at time 2 in the data frame
    i = z + 2
    #pull values from the dataframe
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
    rain_tomorrow = df.iloc[(i),1]
    
    heuristic_df.iat[z,0] = yesterday
    heuristic_df.iat[z,1] = today
    heuristic_df.iat[z,2] = tomorrow
    heuristic_df.iat[z,3] = False # set guess default to False
    heuristic_df.iat[z,4] = rain_tomorrow
    
    #example hueristic
    if today > 0.0 and yesterday > 0.0:
        heuristic_df.iat[z,3] = True
        
    if heuristic_df.iat[z,3] == heuristic_df.iat[z,4]:
        heuristic_df.iat[z,5] = True
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,6] = True #true positive
        else:
            heuristic_df.iat[z,8] = True #true negative
    else:
        heuristic_df.iat[z,5] = False
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,7] = True #false positive
        else:
            heuristic_df.iat[z,9] = True #false negative

In [38]:
heuristic_df.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
0,0.47,0.59,0.42,True,True,True,True,False,False,False
1,0.59,0.42,0.31,True,True,True,True,False,False,False
2,0.42,0.31,0.17,True,True,True,True,False,False,False
3,0.31,0.17,0.44,True,True,True,True,False,False,False
4,0.17,0.44,0.41,True,True,True,True,False,False,False


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
iris = load_iris()
x, y = iris.data, iris.target
import warnings


In [40]:
warnings.filterwarnings("ignore", category=FutureWarning)

X_train, X_test, y_train, y_test = train_test_split(df, df['RAIN'], test_size=0.2,random_state=123)

In [41]:
X_train.head()

,DATE,PRCP,TMAX,TMIN,RAIN
19109,2000-04-26,0.00,62,41,False
13919,1986-02-09,0.00,45,29,False
12367,1981-11-10,0.04,60,38,True
15279,1989-10-31,0.00,54,42,False
20620,2004-06-15,0.00,69,49,False


In [42]:
X_test.head()

,DATE,PRCP,TMAX,TMIN,RAIN
14025,1986-05-26,0.02,68,56,True
22055,2008-05-20,0.19,59,49,True
10213,1975-12-18,0.00,45,27,False
16935,1994-05-14,0.14,64,46,True
15217,1989-08-30,0.00,69,55,False


In [43]:
y_train.head()

19109    False
13919    False
12367     True
15279    False
20620    False
Name: RAIN, dtype: object

In [44]:
y_test.head()

14025     True
22055     True
10213    False
16935     True
15217    False
Name: RAIN, dtype: object

In [55]:
#Accuracy of train set 
TP_TN=X_train['RAIN'].value_counts()
acc=TP_TN/numrows
acc
print("Accuracy:\n\n",acc)

Accuracy:

 False    0.460644
True     0.339309
Name: RAIN, dtype: float64


In [57]:
#Accuracy of test set 
TP_TN=X_test['RAIN'].value_counts()
acc=TP_TN/numrows
acc
print("Accuracy:\n\n",acc)

Accuracy:

 False    0.112685
True     0.087322
Name: RAIN, dtype: float64


In [62]:
# precision of train

num_true_positive=X_train['TMAX'].sum()
num_false_positive_=X_train['TMIN'].sum()
Precision = num_true_positive/(num_true_positive+num_false_positive_)
print("Precision:", Precision)

Precision: 0.5723651857265413


In [63]:
# precision pf test
num_true_positive=X_test['TMAX'].sum()
num_false_positive_=X_test['TMIN'].sum()
Precision = num_true_positive/(num_true_positive+num_false_positive_)
print("Precision:", Precision)

Precision: 0.5716367750806455


In [64]:
# recall of train set
num_false_negative=X_train['RAIN'].sum()
recall=num_true_positive/(num_true_positive+num_false_negative)
print("Recall:", recall)

Recall: 0.9723757962392334


In [65]:
num_false_negative=X_test['RAIN'].sum()
recall=num_true_positive/(num_true_positive+num_false_negative)
print("Recall:", recall)

Recall: 0.9927419066240268


In [66]:
import numpy as np

def sse(y_true, y_pred):
    '''returns sum of squared errors (actual vs model)'''
    squared_errors = (y_true - y_pred) ** 2
    return np.sum(squared_errors)


sse(y_true=heuristic_df['rain_tomorrow'].astype('int'), y_pred=heuristic_df['guess'].astype('int'))

8390

In [67]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true=heuristic_df['rain_tomorrow'].astype('int'), y_pred=heuristic_df['guess'].astype('int'))


0.3283885866374418

In [68]:
#Calculate the mean
mean_PRCP=X_train['PRCP'].sum()/X_train['PRCP'].value_counts() 
deviation1=X_train['PRCP']-mean_PRCP
deviation2=deviation1**2
print("Mean:{} deviation1: {} deviation2: {}".format(mean_PRCP,deviation1,deviation2))


Mean:0.00       0.184292
0.01       2.903521
0.02       3.873089
0.03       5.490962
0.04       6.214699
           ...     
2.64    2168.930000
3.77    2168.930000
2.04    2168.930000
2.06    2168.930000
1.91    2168.930000
Name: PRCP, Length: 198, dtype: float64 deviation1: 0.00        0.285708
0.01             NaN
0.02             NaN
0.03             NaN
0.04             NaN
              ...   
25546.00         NaN
25547.00         NaN
25548.00         NaN
25549.00         NaN
25550.00         NaN
Name: PRCP, Length: 20636, dtype: float64 deviation2: 0.00        0.081629
0.01             NaN
0.02             NaN
0.03             NaN
0.04             NaN
              ...   
25546.00         NaN
25547.00         NaN
25548.00         NaN
25549.00         NaN
25550.00         NaN
Name: PRCP, Length: 20636, dtype: float64
